## Example loading of CytoFM weights into a ViT
- Code is based on that found in the iBOT Github: https://github.com/bytedance/ibot
- Need to have pulled iBOT code to run this notebook!

In [1]:
import sys
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch

#!git clone https://github.com/bytedance/ibot
sys.path.append("/home/hoan-ngo/Documents/AI4Thyroid/Models/ibot/")


from models import VisionTransformer, vit_tiny, vit_small, vit_base, vit_large
from models import SwinTransformer, swin_tiny, swin_small, swin_base



#%load_ext autoreload
#%autoreload 2

/home/hoan-ngo/miniconda3/envs/torch/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hoan-ngo/miniconda3/envs/torch/lib/python3.14/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/hoan-ngo/miniconda3/envs/torch/lib/python3.14/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
!nvidia-smi

Sun Dec 28 20:46:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.95.05              Driver Version: 580.95.05      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060        Off |   00000000:01:00.0  On |                  N/A |
|  0%   57C    P5             25W /  170W |     657MiB /  12288MiB |     16%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Version (built with): {torch.version.cuda}")

PyTorch Version: 2.9.1+cu130
CUDA Available: True
CUDA Version (built with): 13.0


In [6]:
# path to model weights
model_weights = '/home/hoan-ngo/Documents/AI4Thyroid/Models/CytoFM_Weights/cytofm_weights.pth'

In [7]:
# load weights
ck = torch.load(model_weights, map_location=torch.device('cpu'), weights_only=False)

In [8]:
# load teacher weights into vit_base (configured in ibot code)
output_dict = dict(state_dict=dict())
has_backbone = False
for key, value in ck['teacher'].items():
    #print(key)
    if key.startswith('backbone'):
        output_dict['state_dict'][key[9:]] = value
        has_backbone = True
    elif key.startswith('module.backbone'):
        output_dict['state_dict'][key[16:]] = value
        has_backbone = True
vit = vit_base(patch_size=16).cuda()
vit.load_state_dict(output_dict['state_dict'], strict=True)
vit.eval();
for p in vit.parameters():
    p.requires_grad = False

In [9]:
# check model loaded correctly and runs on an image

img = torch.randn([1, 3, 224, 224])

with torch.no_grad():
    features = vit(img.cuda())
features.shape

torch.Size([1, 768])